In [28]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import keras
import tensorflow as tf
from keras.layers import Input, Dense, Reshape
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

# Link to Google Drive in order to store and use certain files
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive


# Testing out the target model

In [ ]:
labels = ['Created with AI', 'Not Created with AI']

# Included in the program
def preprocess_image(image, img_size=256):
    img = cv2.resize(image, (img_size, img_size))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Load image, feel free to uncomment these (and comment the normal.png) to test the AI model.

# AI Generated
# image_name = "FGSM_Data/test1.png"

# AI Generated
# image_name = "FGSM_Data/test2.png"

# Not AI Generated
# image_name = "FGSM_Data/test3.png"

# Not AI Generated
# image_name = "FGSM_Data/test4.png"

# Our image to turn into adversarial, you can also use other 256x256 images.
image_name = "FGSM_Data/normal.png"

original_image = np.array(Image.open(image_name).convert('RGB'))
model = keras.models.load_model("FGSM_Data/ai_prediction_model.h5")
model.trainable = False

# Preprocess image and predict the result
image = preprocess_image(original_image)
results = model.predict(image)

# Visualize Image and result
plt.imshow(original_image)
plt.title("Prediction Result: " + labels[np.argmax(results)] + "\n" + str(results[0][0]) + " vs " + str(results[0][1]))

# Create our own adversarial image

In [30]:
# Explaination on how the model works
# The model will first convert the image into BGR format,
# and then it will normalize the images to [0, 1] range

# Converts image from RGB to BGR, then normalize to [0, 1] range, and converts it to a tensorflow object
def preprocess_image(image):
    img = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    img = img / 255.0
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (256, 256))
    img = img[None, ...]
    return img

# FGSM
def fgsm(input_image, input_label):
    # One-hot label, aka 1 and 0s only
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    with tf.GradientTape() as tape:
        # Record the gradient
        tape.watch(input_image)
        # Make a prediction
        prediction = model(input_image)
        # Calculate the loss
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

# Load original image, model
original_image = np.array(Image.open("FGSM_Data/normal.png").convert('RGB'))
adv_image = np.array(Image.open("FGSM_Data/normal.png").convert('RGB'))
model = keras.models.load_model("FGSM_Data/ai_prediction_model.h5")

# Create a tensorflow label, where we want the output to NOT be 'Created with AI'
y_adx = tf.cast(np.array([[1,0]]), tf.float32)
adv_image = preprocess_image(adv_image)

# Slowly add the perturbations for 3 iterations (gradient)
epsilon = 0.005
for i in range(3):
    perturbations = fgsm(adv_image, y_adx)
    adv_image += epsilon * perturbations

# Convert our image back to RGB, into [0-255] range and save the image
adv_image = adv_image.numpy()
adv_image = cv2.normalize(adv_image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8U)
adv_image = adv_image.squeeze(axis=0)
adv_image = cv2.cvtColor(adv_image, cv2.COLOR_BGR2RGB)
im = Image.fromarray(adv_image)
im.save("FGSM_Data/adversarial.png")

# Visualize our adversarial image

In [ ]:
labels = ['Created with AI', 'Not Created with AI']

original_image = np.array(Image.open("FGSM_Data/normal.png").convert('RGB'))
adv_image = np.array(Image.open("FGSM_Data/adversarial.png").convert('RGB'))
model = keras.models.load_model("FGSM_Data/ai_prediction_model.h5")

# Predict result
original_image_preprocessed = preprocess_image(original_image)
original_image_prediction = model.predict(original_image_preprocessed)
adv_image_preprocessed = preprocess_image(adv_image)
adv_image_prediction = model.predict(adv_image_preprocessed)

# Plot the first image
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(1, 2, 1)
plt.imshow(original_image)
plt.title("Original Image"+ "\n" + "Prediction Result: " + labels[np.argmax(original_image_prediction)] + "\n" + str(original_image_prediction[0][0]) + " vs " + str(original_image_prediction[0][1]))

# Plot the second image
fig.add_subplot(1, 2, 2)
plt.imshow(adv_image)
plt.title("Adversarial Image" + "\n" + "Prediction Result: " + labels[np.argmax(adv_image_prediction)] + "\n" + str(adv_image_prediction[0][0]) + " vs " + str(adv_image_prediction[0][1]))